In [1]:
import os
import openai
import langchain

In [2]:
openai.__version__

'0.28.1'

In [8]:
usso_token = 'eyJhbGciOiJFUzI1NiIsImVudiI6InByb2QiLCJraWQiOiJnX0hsUlpnWGRzMlNFcUVFUGVzZmdiUDBaRWxIV2tiaFJ5SUd4alRKMWNrIiwidHlwIjoiSldUIiwidmVyIjoiMS4wIn0.eyJjbGllbnRfaWQiOiJnZW5haS1hcGkudWJlcmludGVybmFsLmNvbSIsImVtYWlsIjoiYWphaXN3MjZAZXh0LnViZXIuY29tIiwiZXhwIjoxNzAwNTI1OTU3LCJpYXQiOjE3MDA0NTM2NTcsImlzcyI6InNwaWZmZTovL3Vzc28udXBraS5jYSIsImp0aSI6IjI1NWNiZTgwLThlZTQtNDRkNS05MGZhLTFhZDIzNWYzNjY2MCIsInBsY3kiOiJWeElSY1QyZ1VieTdKYXVMTXRuT3B0Zml5WkovdW9NcnBjSjcxR0xHcmU2ZTNZL2wrakVjeWJiV0lWNGJqeDhPYWlWY1FlblFzeE1HTzF6OW03TFpvUm5HMCt3TlJKeENaVUcxdFBvUjE0Y0FNeGY4b3pTYjkxUE94dTNhMWlJbFdrajJzYVVrUUlqemMyYTlZREJTeEJ2SzhnPT0iLCJwbGN5X2tleSI6ImtleS11c3NvLXBsY3ktMTEwOTE4LnBlbSIsInJvbGVzIjoiY2xpZW50Iiwic3ViIjoic3BpZmZlOi8vcGVyc29ubmVsLnVwa2kuY2EvZWlkLzk5OTAwMDI2ODM4MCIsInRlbmFuY3kiOiJ1YmVyL3Byb2R1Y3Rpb24iLCJ0b2tlbiI6IjJkM2I5MDJlLTA2M2YtMTFlYy05N2RjLWIwMjYyODY4N2JjMCIsInR5cGUiOiJvZmZsaW5lIiwidXVpZCI6ImNjMTA0YzcwLWIxNTYtNGM1NS05ZmE1LTY5MzJmNzM2ZDA5YyJ9.iioK5pZFlU6YNPfheCef6DUDuUCifiUctS62ubsuhEp9tm4RXCEVXTMS_7pc2snYr-mfsu6xunO63w1rpuINdg'
os.environ["OPENAI_API_KEY"] = usso_token
openai.api_base = 'https://genai-api.uberinternal.com/v1'

In [4]:
from langchain.document_loaders import (
    UnstructuredCSVLoader,
    UnstructuredHTMLLoader,
    UnstructuredImageLoader,
    UnstructuredWordDocumentLoader,
    PythonLoader,
    PyPDFLoader,
    JSONLoader,
    Docx2txtLoader,
    ToMarkdownLoader,
    UnstructuredFileLoader
)
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter
)

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=0,
)


In [5]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI()

def summarizerGen(_verbose=False):
    map_template = """Write a concise yet compelling summary of the following:
    
    
    "{text}"
    
    
    CONCISE SUMMARY:"""
    
    combine_template = """The following is a set of summaries of different part of a big document:
    
    "{text}"
    
    Take these and distill it into a final but consolidated, impactful and coherent summary. 
    The FINAL SUMMARY shall have these points : 
    1. What is the main theme of main document?
    2. What the whole content is about ?
    3. What inference we can make out of it ?
    FINAL SUMMARY:"""
    
    map_prompt = PromptTemplate.from_template(map_template)
    combine_prompt = PromptTemplate.from_template(combine_template)
    
    summarizerChain = load_summarize_chain(
        llm=llm,
        chain_type='map_reduce',
        map_prompt=map_prompt,
        combine_prompt=combine_prompt,
        verbose=_verbose
    )
    return summarizerChain

In [6]:
def customLoaderGen(file_path, file_type):
    print(file_type.lower())
    try:
        if file_type.lower()=='csv':
            customLoader = CSVLoader(file_path=file_path)
        if file_type.lower()=='txt':
            customLoader = TextLoader(file_path=file_path)
        if file_type.lower()=='docx':
            customLoader = Docx2txtLoader(file_path=file_path)
        if file_type.lower()=='pdf':
            customLoader = PyPDFLoader(file_path=file_path)
        if file_type.lower()=='html':
            customLoader = UnstructuredHTMLLoader(file_path=file_path)
        if file_type.lower() in ('md', 'markdown'):
            customLoader = ToMarkdownLoader(file_path=file_path)
    except:
        print('using file loader')
        customLoader = UnstructuredFileLoader(file_path=file_path)
    return customLoader

In [7]:
summarizerChain = summarizerGen()

In [8]:
file_path = '../langchain/data/mixed_data/MachineLearning-Lecture01.pdf'
file_type = 'PDF'

In [9]:
%%time
customLoader = customLoaderGen(file_path, file_type)
data = customLoader.load_and_split(text_splitter=recursive_splitter)
len(data)

CPU times: user 178 ms, sys: 10.7 ms, total: 188 ms
Wall time: 187 ms


22

In [10]:
%%time
summarizerChain.run(data[5:10])

CPU times: user 365 ms, sys: 11.6 ms, total: 376 ms
Wall time: 8.27 s


'The main theme of the document is a machine learning course that emphasizes academic integrity and encourages collaboration. The content covers various aspects of the course, including online resources, homework assignments, study groups, and the importance of MATLAB. It also mentions a midterm exam and a research project. From the document, we can infer that students are expected to complete their homework independently, utilize the provided resources, and collaborate responsibly. The goal of the course is to produce publishable research in the field of machine learning.'

In [11]:
data[5:10]

[Document(page_content="So all right, online resources. The class has a home page, so it's in on the handouts. I \nwon't write on the chalkboard — http:// cs229.stanford.edu. And so when there are \nhomework assignments or things like that, we  usually won't sort of — in the mission of \nsaving trees, we will usually not give out many handouts in class. So homework \nassignments, homework solutions will be posted online at the course home page.  \nAs far as this class, I've also written, a nd I guess I've also revised every year a set of \nfairly detailed lecture notes that cover the te chnical content of this  class. And so if you \nvisit the course homepage, you'll also find the detailed lecture notes that go over in detail \nall the math and equations and so on  that I'll be doing in class.  \nThere's also a newsgroup, su.class.cs229, also written on the handout. This is a \nnewsgroup that's sort of a forum for people in  the class to get to  know each other and \nhave whatever disc